# 收获和价值的计算(马尔可夫奖励过程)
- 收获(return) 收获是对应于状态序列中的某一时刻的 状态的，计算从该状态开始直至结束还能获得的累积奖励 
$$G _ { t } = R _ { t + 1 } + \gamma R _ { t + 2 } + \ldots = \sum _ { k = 0 } ^ { \infty } \gamma ^ { k } R _ { t + k + 1 }$$
- 价值(value) 是马尔科夫奖励过程中状态收获的期望
$$ v ( s ) = E [ G _ { t } | S _ { t } = s ] $$
- 价值函数展开一窥奥秘
$$  \left. \begin{array}{l}{ v ( s ) } \\ { = E [ G _ { t } | S _ { t } = s ] }\\{ = E [ R _ { t + 1 } + \gamma v ( S _ { t + 1 } ) | S _ { t } = s ] } \\ { = R_s + \gamma v( s ^ { \prime } ) }\end{array} \right.$$
- 价值函数=>(贝尔曼方程) 等于此时刻的奖励值和下一个状态的价值。下一个状态的价值就是透过此时刻到下一时刻全部状态的平均价值
$$ v ( s ) = R _ { s } + \gamma \sum _ { s ^ { \prime } \in S } P _ { s s ^ { \prime } } v ( s ^ { \prime } ) $$
- 用矩阵形式计算价值。可以直接求解。复杂度O(n^3)
$$ \left. \begin{array}{l}{ v = R + \gamma P v }\\{ ( 1 - \gamma P ) v = R }\\{ v = ( 1 - \gamma P ) ^ { - 1 } R }\end{array} \right. $$
> 相较于值计算Return， Return只考虑了某些特殊的情况，并没有从必然性来考虑。所以求他的期望是比较准确的
> 这种马尔可夫奖励过程不涉及选择动作，预定了的概率存在，我们只是计算出了每种状态的价值->改进为马尔可夫决策过程(MDP)

In [5]:
import numpy as np
# 这些状态有:第 一节课(C1)、第二节课(C2)、第三节课(C3)、泡吧中(Pub)、通过考试(Pass)、浏览手机 (FB)、以及休息退出(Sleep)共 7 个状态
# 索引到状态名的对应

i_to_n = {}
i_to_n["0"] = "C1"
i_to_n["1"] = "C2"
i_to_n["2"] = "C3"
i_to_n["3"] = "Pass"
i_to_n["4"] = "Pub"
i_to_n["5"] = "FB"
i_to_n["6"] = "Sleep"

n_to_i={}# 状态名到索引的字典
for i, name in zip(i_to_n.keys(), i_to_n.values()):
    n_to_i[name] = int(i)

# 此时我们预定有概率转移矩阵的，因为我们的目标仅仅是计算收获Return和价值Value
Pss=[# 状态转移概率矩阵
    [ 0.0, 0.5, 0.0, 0.0, 0.0, 0.5, 0.0 ],
    [ 0.0, 0.0, 0.8, 0.0, 0.0, 0.0, 0.2 ],
    [ 0.0, 0.0, 0.0, 0.6, 0.4, 0.0, 0.0 ],
    [ 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0 ],
    [ 0.2, 0.4, 0.4, 0.0, 0.0, 0.0, 0.0 ],
    [ 0.1, 0.0, 0.0, 0.0, 0.0, 0.9, 0.0 ],
    [ 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0 ]
]

Pss = np.array(Pss)
# 奖励值、对应的是状态。也就是每个状态给他一个奖励值
rewards = [-2, -2, -2, 10, 1, -1 ,0]
gamma = 0.5

# 计算某一条链的累积奖励值Return
def compute_return(start_index = 0, chain = None, gamma = 0.5):
    retrn, power, gamma = 0.0, 0, gamma
    for i in range(start_index, len(chain)):
        retrn += np.power(gamma, power) * rewards[n_to_i[chain[i]]]
        power += 1
    return retrn

# 使用矩阵计算价值Value
# 相较于值计算Return， Return只考虑了某些特殊的情况，并没有从必然性来考虑。所以求他的期望是比较准确的
# 这种马尔可夫奖励过程不涉及选择动作，预定了的概率存在，我们只是计算出了每种状态的价值->改进为马尔可夫决策过程(MDP)
def compute_value(Pss, rewards, gamma = 0.999):
    # 将rewards转为numpy数组并修改为列向量的形式
    rewards = np.array(rewards).reshape(-1, 1)
    # np.eye(7,7)为 单 位 矩 阵，inv方 法 为 求 矩 阵 的 逆
    values = np.dot(np.linalg.inv(np.eye(7, 7) - gamma * Pss), rewards)
    return values

In [4]:
chains =[
["C1", "C2", "C3", "Pass", "Sleep"],
["C1", "FB", "FB", "C1", "C2", "Sleep"],
["C1", "C2", "C3", "Pub", "C2", "C3", "Pass", "Sleep"], ["C1", "FB", "FB", "C1", "C2", "C3", "Pub", "C1", "FB",\
"FB", "FB", "C1", "C2", "C3", "Pub", "C2", "Sleep"] ]

compute_return(0, chains[3], gamma=0.5)


-3.196044921875

In [6]:
compute_value(Pss, rewards, gamma=0.99999)

array([[-12.54073351],
       [  1.45690179],
       [  4.32117045],
       [ 10.        ],
       [  0.80308417],
       [-22.53857963],
       [  0.        ]])

# 马尔可夫决策过程


In [ ]:
# 状态数变 成了 5 个，为了方便理解，我们把这五个状态分别命名为:‘浏览手机中’,‘第一节课’,‘第二 节课’,‘第三节课’,‘休息中’;
# 行为总数也是 5 个，但具体到某一状态则只有 2 个可能的行 为，这 5 个行为分别命名为:‘浏览手机’,‘学习’,‘离开浏览’,‘泡吧’,‘退出学习’

